In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

root = '3-condition-splited-data'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),         
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = ImageFolder(root=root + '/train', transform=transform)
test_data = ImageFolder(root=root + '/test', transform=transform)
val_data = ImageFolder(root=root+"/validation",transform=transform)

batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

C:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import torch
import torch.nn as nn

class ANN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, num_classes):
        super(ANN, self).__init__()
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.batch1 = nn.BatchNorm1d(hidden_size1)
        self.drop1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.batch2 = nn.BatchNorm1d(hidden_size2)
        self.drop2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.batch3 = nn.BatchNorm1d(hidden_size3)
        self.drop3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.relu4 = nn.ReLU()
        self.batch4 = nn.BatchNorm1d(hidden_size4)
        self.drop4 = nn.Dropout(0.5)

        self.fc5 = nn.Linear(hidden_size4, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.batch1(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batch2(x)
        x = self.drop2(x)

        x = self.fc3(x)
        x = self.relu3(x)
        x = self.batch3(x)
        x = self.drop3(x)

        x = self.fc4(x)
        x = self.relu4(x)
        x = self.batch4(x)
        x = self.drop4(x)

        x = self.fc5(x)

        return x


In [3]:
import torch.optim as optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_size = 256 * 256 * 3

#12288x256 and 196608x256
model = ANN(input_size,256,128,64,32,len(train_data.classes)).to(device)

losS = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 30


for epoch in range(epochs):
    model.train()

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = losS(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'epoch : [{epoch + 1}/{epochs}], train loss : {loss.item()}')

    model.eval()
    total_loss = 0.0
    acc = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            total_loss += losS(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            acc += (predicted == labels).sum().item()

    total_loss /= len(val_loader.dataset)
    accuracy = acc / len(val_loader.dataset) * 100.0

    print(f'test loss : {total_loss} , test accuracy: {accuracy}')


epoch : [1/30], train loss : 0.13824684917926788
test loss : 0.009987676590681076 , test accuracy: 96.0
epoch : [2/30], train loss : 0.9354028105735779
test loss : 0.004844829075570618 , test accuracy: 98.71428571428571
epoch : [3/30], train loss : 0.060593292117118835
test loss : 0.003659025402739644 , test accuracy: 98.85714285714286
epoch : [4/30], train loss : 0.07763414829969406
test loss : 0.003015698213795466 , test accuracy: 99.14285714285714
epoch : [5/30], train loss : 0.046810805797576904
test loss : 0.003319491389141019 , test accuracy: 98.71428571428571
epoch : [6/30], train loss : 0.08480498194694519
test loss : 0.0033034750820869316 , test accuracy: 97.71428571428571
epoch : [7/30], train loss : 0.011975795030593872
test loss : 0.00252154443146927 , test accuracy: 98.85714285714286
epoch : [8/30], train loss : 0.007681908551603556
test loss : 0.0022912223933131567 , test accuracy: 98.71428571428571
epoch : [9/30], train loss : 0.037671614438295364
test loss : 0.003777346

In [4]:
torch.save(model.state_dict(), "ann-3-fit.pth")